In [1]:
from libraries import lib
from libraries.NeuralStates import *
import qutip as qt
import torch.nn as nn 
import torch
import numpy as np
import matplotlib.pyplot as plt
from kan import *

In [2]:
def amp_phase(nn_output):
    return nn_output[:, 0] * torch.exp(1.j * 2 * np.pi * nn_output[:, 1])
def bitflip_x(x, N, flips):
    new_x = x
    for _ in range(flips):
        new_x = x ^ (1 << npr.randint(0, N))
    return new_x

In [5]:
N, J, Gamma = 6, 1, 1

In [7]:
true_gse = lib.ground_state_energy(Gamma, N)
print(true_gse)

-7.727406610312546


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
input = lib.generate_input_torch(N)
def amp_phase(nn_output):
    return nn_output[:, 0] * torch.exp(1.j * 2 * np.pi * nn_output[:, 1])

cpu


In [13]:
kan_model = KAN(width=[N, N, 2], device=device)

checkpoint directory created: ./model
saving model version 0.0


In [14]:
mlp_model = nn.Sequential(
    nn.Linear(N, 5 * N),
    nn.Sigmoid(),
    nn.Linear(5 * N, 2),
    nn.Sigmoid(),
)

In [18]:
num_epochs = 5
data_rate = 1
num_samples = 128

In [16]:
len(amp_phase(kan_model(input)))

64

In [17]:
len(amp_phase(mlp_model(input)))

64

In [19]:
vars = (N, J, Gamma)
loss_fn = lib.TFIM_expectation_using_locals
epochs = []
loss_data = []
optimizer = LBFGS(kan_model.parameters(), lr=1, history_size=10, line_search_fn="strong_wolfe", tolerance_grad=1e-32, tolerance_change=1e-32, tolerance_ys=1e-32)
for epoch in range(num_epochs):
    print('epoch', epoch)
    def closure():
        optimizer.zero_grad()
        loss = loss_fn(MHNeuralState(N, kan_model, amp_phase, lambda x : bitflip_x(x, N, 1), 2 ** (N - 1), num_samples), N, J, Gamma, model, amp_phase)
        loss.backward()
        print('loss', loss)
        loss_data.append(loss)
        return loss

    optimizer.step(closure)
    epochs.append(epoch)
    # loss_data.append(loss_fn(model(input), vars, amp_phase).item())

# find groud state
gs = lib.model_to_ground_state(kan_model, input, amp_phase)

epoch 0


c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:813: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1808.)
  self.subnode_actscale.append(torch.std(x, dim=0).detach())
c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:823: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1808.)
  input_range = torch.std(preacts, dim=0) + 0.1
c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:824: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (inpu

loss tensor(-4.6566, grad_fn=<SelectBackward0>)
loss tensor(-4.9741, grad_fn=<SelectBackward0>)
loss tensor(-5.1721, grad_fn=<SelectBackward0>)
loss tensor(-5.3390, grad_fn=<SelectBackward0>)
loss tensor(-5.7390, grad_fn=<SelectBackward0>)
loss tensor(-2.0426, grad_fn=<SelectBackward0>)
loss tensor(-5.6930, grad_fn=<SelectBackward0>)
loss tensor(-5.4972, grad_fn=<SelectBackward0>)
loss tensor(-5.4480, grad_fn=<SelectBackward0>)
loss tensor(-5.2418, grad_fn=<SelectBackward0>)
loss tensor(-5.6882, grad_fn=<SelectBackward0>)
epoch 1
loss tensor(-5.3451, grad_fn=<SelectBackward0>)
loss tensor(-1.0916, grad_fn=<SelectBackward0>)
loss tensor(-5.6871, grad_fn=<SelectBackward0>)
loss tensor(-2.7673, grad_fn=<SelectBackward0>)
loss tensor(-5.9888, grad_fn=<SelectBackward0>)
loss tensor(-5.2421, grad_fn=<SelectBackward0>)
loss tensor(-6.3193, grad_fn=<SelectBackward0>)
loss tensor(-4.1191, grad_fn=<SelectBackward0>)
loss tensor(-6.3012, grad_fn=<SelectBackward0>)
loss tensor(-5.7932, grad_fn=<Se

In [20]:
loss_fn(MHNeuralState(N, kan_model, amp_phase, lambda x : bitflip_x(x, N, 1), 2 ** (N - 1), num_samples), N, J, Gamma, model, amp_phase)

c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:813: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1808.)
  self.subnode_actscale.append(torch.std(x, dim=0).detach())
c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:823: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1808.)
  input_range = torch.std(preacts, dim=0) + 0.1
c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:824: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (inpu

tensor(-7.8717, grad_fn=<SelectBackward0>)

In [21]:
loss_data

[tensor(-4.6566, grad_fn=<SelectBackward0>),
 tensor(-4.9741, grad_fn=<SelectBackward0>),
 tensor(-5.1721, grad_fn=<SelectBackward0>),
 tensor(-5.3390, grad_fn=<SelectBackward0>),
 tensor(-5.7390, grad_fn=<SelectBackward0>),
 tensor(-2.0426, grad_fn=<SelectBackward0>),
 tensor(-5.6930, grad_fn=<SelectBackward0>),
 tensor(-5.4972, grad_fn=<SelectBackward0>),
 tensor(-5.4480, grad_fn=<SelectBackward0>),
 tensor(-5.2418, grad_fn=<SelectBackward0>),
 tensor(-5.6882, grad_fn=<SelectBackward0>),
 tensor(-5.3451, grad_fn=<SelectBackward0>),
 tensor(-1.0916, grad_fn=<SelectBackward0>),
 tensor(-5.6871, grad_fn=<SelectBackward0>),
 tensor(-2.7673, grad_fn=<SelectBackward0>),
 tensor(-5.9888, grad_fn=<SelectBackward0>),
 tensor(-5.2421, grad_fn=<SelectBackward0>),
 tensor(-6.3193, grad_fn=<SelectBackward0>),
 tensor(-4.1191, grad_fn=<SelectBackward0>),
 tensor(-6.3012, grad_fn=<SelectBackward0>),
 tensor(-5.7932, grad_fn=<SelectBackward0>),
 tensor(-6.0891, grad_fn=<SelectBackward0>),
 tensor(-5

In [23]:
Gamma = 0.1

In [24]:
true_gse = lib.ground_state_energy(Gamma, N)
print(true_gse)

-6.0150096436544


In [28]:
kan_model_2 = KAN(width=[N, N, 2], device=device)

checkpoint directory created: ./model
saving model version 0.0


In [29]:
vars = (N, J, Gamma)
loss_fn = lib.TFIM_expectation_using_locals
epochs = []
loss_data = []
optimizer = LBFGS(kan_model_2.parameters(), lr=1, history_size=10, line_search_fn="strong_wolfe", tolerance_grad=1e-32, tolerance_change=1e-32, tolerance_ys=1e-32)
for epoch in range(num_epochs):
    print('epoch', epoch)
    def closure():
        optimizer.zero_grad()
        loss = loss_fn(MHNeuralState(N, kan_model_2, amp_phase, lambda x : bitflip_x(x, N, 1), 2 ** (N - 1), num_samples), N, J, Gamma, kan_model_2, amp_phase)
        loss.backward()
        print('loss', loss)
        loss_data.append(loss)
        return loss

    optimizer.step(closure)
    epochs.append(epoch)
    # loss_data.append(loss_fn(model(input), vars, amp_phase).item())

# find groud state
gs = lib.model_to_ground_state(kan_model_2, input, amp_phase)

epoch 0


c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:813: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1808.)
  self.subnode_actscale.append(torch.std(x, dim=0).detach())
c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:823: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1808.)
  input_range = torch.std(preacts, dim=0) + 0.1
c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:824: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (inpu

loss tensor(-0.2644, grad_fn=<SelectBackward0>)
loss tensor(-0.5371, grad_fn=<SelectBackward0>)
loss tensor(-0.6150, grad_fn=<SelectBackward0>)
loss tensor(-0.7231, grad_fn=<SelectBackward0>)
loss tensor(-1.2846, grad_fn=<SelectBackward0>)
loss tensor(-1.6233, grad_fn=<SelectBackward0>)
loss tensor(-2.1897, grad_fn=<SelectBackward0>)
loss tensor(-2.2173, grad_fn=<SelectBackward0>)
loss tensor(-2.2149, grad_fn=<SelectBackward0>)
loss tensor(-2.4127, grad_fn=<SelectBackward0>)
loss tensor(-2.4947, grad_fn=<SelectBackward0>)
loss tensor(-2.3667, grad_fn=<SelectBackward0>)
loss tensor(-2.5072, grad_fn=<SelectBackward0>)
loss tensor(-2.6519, grad_fn=<SelectBackward0>)
loss tensor(-2.0092, grad_fn=<SelectBackward0>)
loss tensor(-2.7574, grad_fn=<SelectBackward0>)
loss tensor(-1.8003, grad_fn=<SelectBackward0>)
loss tensor(-1.6991, grad_fn=<SelectBackward0>)
loss tensor(-2.9368, grad_fn=<SelectBackward0>)
loss tensor(-3.1083, grad_fn=<SelectBackward0>)
loss tensor(-2.9081, grad_fn=<SelectBack

In [31]:
loss_fn(MHNeuralState(N, kan_model_2, amp_phase, lambda x : bitflip_x(x, N, 1), 2 ** (N - 1), num_samples), N, J, Gamma, kan_model_2, amp_phase)

c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:813: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1808.)
  self.subnode_actscale.append(torch.std(x, dim=0).detach())
c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:823: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1808.)
  input_range = torch.std(preacts, dim=0) + 0.1
c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:824: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (inpu

tensor(-3.9516, grad_fn=<SelectBackward0>)

In [32]:
gs

Quantum object: dims=[[64], [1]], shape=(64, 1), type='ket', dtype=Dense
Qobj data =
[[-0.23204908-0.02012957j]
 [-0.12277526-0.04724472j]
 [-0.10423242+0.02404439j]
 [-0.08871173-0.00979267j]
 [-0.08819614+0.01646521j]
 [-0.01706136-0.00103092j]
 [ 0.02484111-0.01074945j]
 [-0.01812894+0.00238609j]
 [-0.12355423+0.02413674j]
 [-0.05533284-0.00480766j]
 [-0.02141453+0.01131867j]
 [-0.03531292+0.00753819j]
 [ 0.01419587-0.00649789j]
 [ 0.01476056-0.00332174j]
 [ 0.05082028-0.03607117j]
 [ 0.04365736-0.02122965j]
 [-0.15410417-0.01075705j]
 [-0.02665552-0.00839565j]
 [-0.02242329+0.00359524j]
 [-0.00810817-0.00129486j]
 [-0.00732632+0.00168221j]
 [ 0.05612388-0.0037317j ]
 [ 0.07839926-0.03149233j]
 [ 0.05943369-0.01034554j]
 [-0.03042705+0.00539996j]
 [ 0.04762545+0.00177914j]
 [ 0.06086858-0.02148112j]
 [ 0.04933832-0.00557357j]
 [ 0.07703555-0.04014205j]
 [ 0.10479107-0.04004547j]
 [ 0.10738856-0.06629593j]
 [ 0.19050989-0.09818576j]
 [-0.10310572+0.02362342j]
 [-0.05462633-0.00379136